# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('output_data/cities.csv')
city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,17.51,67,100,1.79,US,1655038735
1,Hilo,19.7297,-155.0900,25.16,95,75,0.89,US,1655038891
2,Busselton,-33.6500,115.3333,14.90,62,100,13.73,AU,1655038813
3,Srednekolymsk,67.4500,153.6833,12.54,38,17,3.92,RU,1655038894
4,Hithadhoo,-0.6000,73.0833,28.54,70,100,2.16,MV,1655038895


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
humidity_loc_df= city_df[['Lat', 'Lng']]
weight = city_df['Humidity']
fig = gmaps.figure(zoom_level=2,center=(17.6078,8.0817))
heatmap_layer = gmaps.heatmap_layer(humidity_loc_df,weights=weight,max_intensity=250,point_radius=4,dissipating=False)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = city_df.loc[(city_df['Max Temp']>=22)&(city_df['Max Temp']<=27)&(city_df['Cloudiness']<=3)&(city_df['Wind Speed']<=3),:]
hotel_df = ideal_df.dropna(how='any')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,Saint George,37.1041,-113.5841,26.08,20,2,1.34,US,1655038702
134,Jiexiu,37.0244,111.9125,22.97,19,0,1.94,CN,1655038992
168,Robertson,36.5167,-86.8667,24.38,85,0,2.57,US,1655039015
254,Novorzhev,57.0294,29.3338,23.36,45,1,2.34,RU,1655039078
450,Yucaipa,34.0336,-117.0431,26.33,45,0,0.45,US,1655039229
484,Manjacaze,-24.7117,33.8828,24.10,44,0,1.28,MZ,1655039258
537,Lewiston,44.1003,-70.2148,23.56,72,0,0.00,US,1655039293


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name']= ""
baseurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "key": g_key,
    "radius": 5000
}

for index,row in hotel_df.iterrows():
    params ['location']=str(row['Lat']) + ',' + str(row['Lng'])
    try:
        response = requests.get(baseurl,params=params)
        response_json = response.json()
    except:
        continue
    
    try:
        hotel_df.loc[index,'Hotel Name'] = response_json['results'][0]['name']
    except (KeyError, IndexError):
        continue
#remove line with no hotel found
hotel_df = hotel_df.loc[hotel_df['Hotel Name']!="",:]

    

In [6]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,Saint George,37.1041,-113.5841,26.08,20,2,1.34,US,1655038702,Best Western Plus Abbey Inn
134,Jiexiu,37.0244,111.9125,22.97,19,0,1.94,CN,1655038992,IU Hotel Jinzhong Yuhua Road
168,Robertson,36.5167,-86.8667,24.38,85,0,2.57,US,1655039015,Hampton Inn Springfield
254,Novorzhev,57.0294,29.3338,23.36,45,1,2.34,RU,1655039078,D. Maloe Nikulino
450,Yucaipa,34.0336,-117.0431,26.33,45,0,0.45,US,1655039229,Yucaipa Regional Park
484,Manjacaze,-24.7117,33.8828,24.10,44,0,1.28,MZ,1655039258,Hotel Gani
537,Lewiston,44.1003,-70.2148,23.56,72,0,0.00,US,1655039293,Hilton Garden Inn Auburn Riverwatch


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<b>Name</b>
<dt>{Hotel Name}</dt>
<b>City</b>
<dt>{City}</dt>
<b>Country</b>
<dt>{Country}</dt>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))